In [1]:
import IEA_model
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader
import utility

from importlib import reload 
reload(utility)
reload(IEA_model)

from sklearn.linear_model import LinearRegression

In [3]:
thold = 1e-3

df = pd.read_csv("gene_list_PCA/THOLD_{}.txt".format(thold))
idx_gene = df["index"].to_numpy()

# data_train = utility.image_gene_dataset(train = True, cv = None)
# data_test = utility.image_gene_dataset(train = False, cv = None)

train_loader = DataLoader(data_train)
validate_loader = DataLoader(data_test)


/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [110]:
idx_gene = np.load("../image_gene/ICA_no_correction/gene_idx_CV_major0.001.npy")

In [182]:

M = IEA_model.IEA(train_loader, validate_loader, idx_gene = idx_gene, gamma = 1., 
                  beta = .1, L = 2, n_patch = None, n_rep = 50, lr = 1e-3).cuda()
# filename = "models/model_L2_cv4_beta0.1_gamma1.0_genes{}".format(len(idx_gene))
# filename = "models/model_L2_cv4_beta0.1_gamma1.0_genes{}".format(len(idx_gene))
filename = "models/model_L2_cv0_beta0.1_gamma1.0_genes{}".format(len(idx_gene))
# filename = M.filename

M.load_state_dict( torch.load(filename) )

<All keys matched successfully>

In [183]:
sid_train, features_train, genes_train = data_train[:]
sid_test, features_test, genes_test = data_train[:]

In [184]:
with torch.no_grad():
    _, _, _, Z_test = M.forward(features_test.cuda(), non_prb = True)
Z_test = Z_test.cpu().data.numpy()


In [185]:
df_Z = pd.DataFrame(data = Z_test, columns = ["IEA{}".format(iii) for iii in range(1, M.L + 1)], index = sid_test)
# df_cov = utility.load_longitudinal()

df = pd.merge(df_Z, df_cov, how = "inner", left_index = True, right_index = True)

In [186]:
df["IEA1"].corr(df["IEA2"])

0.05921595049740498

In [187]:
df["IEA1"].corr(df["BMI_P2"])

0.2291201580188678

In [188]:
df["IEA1"].corr(df["Perc15_Insp_Thirona_P2"])

0.715894488481209

In [189]:
df["IEA1"].corr(df["Pi10_Thirona_P2"])

-0.18529869282932662

In [190]:
df["IEA1"].corr(df["WallAreaPct_seg_Thirona_P2"])

-0.142733243307627

In [191]:
df["IEA2"].corr(df["BMI_P2"])

-0.5891736995784238

In [192]:
df["IEA2"].corr(df["Perc15_Insp_Thirona_P2"])

-0.24268135841382224

In [193]:
df["IEA2"].corr(df["Pi10_Thirona_P2"])

-0.3865214198350481

In [194]:
df["IEA2"].corr(df["WallAreaPct_seg_Thirona_P2"])

-0.46124750441040085